# <center> 🔍 Exploratory Data Analysis </center>

<a id="cont"></a>

## 📑 Table of Contents

<a href="#one">1. Introduction
   - Project Overview

<a href=#two>2. Loading Data
   - Importing libraries
   - Reading the dataset

<a href=#three>3. Exploratory Data Analysis (EDA)</a>

<a href=#five>4. Feature Engineering</a>

<a href=#six>5. Modeling</a>

<a href=#seven>6. Model Performance</a>

<a href=#eight>7. Model Explanations</a>

<a id="one"></a>
## 1. Introduction
<a href=#cont>Back to Table of Contents</a>


---
    
| ⚡ Description: Introducing the project ⚡ |
| :--------------------------- |

---

### *Project Overview*

#### Objective

The aim of this project is to develop a machine learning model that predicts car insurance premiums based on customer demographics. 

#### Context/Background

Accurate premium predictions ensures transparency and fairness in cost assessment for customers 

#### Dataset Overview

| **Source**       | **Description**                          |
|------------------|-------------------------------------------|
| `data/train.csv`  | Training data with target premiums        |
| `data/test.csv`   | Test data for final predictions           |

<a id="two"></a>
## 📂 2. Loading Data
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Loading the data in our environment ⚡ |
| :--------------------------- |

---

### 📚 *Importing libraries*

In [ ]:
#Load the packages 
import warnings
from collections import Counter

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings("ignore")

### 📖 *Reading the dataset*

In [ ]:
data = pd.read_csv("../data/car_insurance_premiums_zar.csv")


#Display the first few rows of the datasets and their shape
display("Dataset", data.head(), data.shape)

In [ ]:
#Get a concise summmary of the data dataset
data.info()

<a id="three"></a>
## 3. 🔍 Exploratory Data Analysis (EDA)
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Exploring the data ⚡ |
| :--------------------------- |

---

In [ ]:
#Check for duplicates in the data set
train_duplicates = data[data.duplicated()]
num_duplicates = len(train_duplicates)
print(f"There are {num_duplicates} duplicates in the data dataset")

In [ ]:
#Check for the distribution of each categorical feature
categorical_columns = data.select_dtypes(include=["object"]).columns
for col in categorical_columns:
    print(f"\nDistribution for {col}")
    print(data[col].value_counts())

* The demographic factors in our dataset are fairly balanced with no significant overrepresentation or underrepresentation of any group meaning that these factors should not introduce any bias into the model training. 

* Vehicle details are also fairly balanced which is ideal for our predictive modelling task.

### Data Quality Checks

#### Education Level

In [ ]:
#Determine the unique values that are in the "Education" column
data["Education_Level"].value_counts()

In [ ]:
#Define minimum age requirements for each education level
education_age_limits = {
    "High School": 18,
    "Higher Certificate": 18,
    "Bachelor's Degree": 18,      
    "Master's Degree": 22,                  
    "Doctorate": 24      
}

# Flag invalid rows where age is too low for the education level
invalid_education_rows = data[data.apply(lambda row: row["Age"] < education_age_limits.get(
    row["Education_Level"], 0), axis=1)]

# Display errors
print(invalid_education_rows)

In [ ]:
# Create a boolean mask for invalid rows where True = Invalid and False = Valid
hue_values = data.index.isin(invalid_education_rows.index)

# Define color palette explicitly as a dictionary
palette = {True: "red", False: "blue"}

# Plot strip plot
plt.figure(figsize=(8, 5))
ax = sns.stripplot(x=data["Education_Level"],
                   y=data["Age"],
                   hue=hue_values,
                   palette=palette,
                   size=8, jitter=True)

# Fix legend labels manually
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, ["Valid", "Invalid"], title="Data Status")

plt.title("Age vs Education Level (Invalid Rows in Red)")
plt.xlabel("Education Level")
plt.ylabel("Age")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Remove invalid rows from the original DataFrame in place
cleaned_data = data[data.apply(
    lambda row: row["Age"] >= education_age_limits.get(row["Education_Level"], float('inf')), axis=1)]

#Check the shape of the resulting dataset
cleaned_data.shape

#### Employment status

In [ ]:
#Find the unique categories in the "Employment_Status" column
cleaned_data["Employment_Status"].value_counts()

In [ ]:
#Find the invalid rows in the dataset according to the mimimum age required for an educational level
employment_age_limits = {
    "Unemployed": (18, 65),
    "Employed": (18, 65), 
    "Self-Employed": (18, 80),
    "Retired": (65, 100)
}

invalid_employment_status = cleaned_data[cleaned_data.apply(lambda row: not (employment_age_limits.get(row["Employment_Status"], (0, 150))[0] <= row["Age"] <= employment_age_limits.get(row["Employment_Status"], (0, 150))[1]), axis = 1)]
invalid_employment_status.shape

In [ ]:
plt.figure(figsize=(8, 5))

# Ensure hue is a 1D Boolean Series
hue_values = cleaned_data.index.isin(
    invalid_employment_status.index)  # Boolean Series

sns.stripplot(x=cleaned_data["Employment_Status"],
              y=cleaned_data["Age"],
              hue=hue_values,  # Fix here
              palette={True: "red", False: "blue"},
              size=8, jitter=True)

plt.title("Age vs Employment Status (Invalid Rows in Red)")
plt.xlabel("Employment Status")
plt.ylabel("Age")
plt.xticks(rotation=45)
plt.legend(title="Invalid Data", labels=["Valid", "Invalid"])
plt.show()

The above graph represents a stacked bar graph which highlights logically invalid data points in red and valid ones in blue.
 
 * There is a high number of red points between the ages of 18-55, which are ages that are too young to be considered retired.
 * There are red points in the "Employed" and 'Unemployed" categories for the 65+ age group because it is uncommon to be employed beyond the age of 65.
 * Anyone beyond the age of 65+ would typically be considered "Retired" even if they have been unemployed their entire lives as they are too old to enter the workforce. 

In [ ]:
# Set target proportions from original distribution
target_distribution = {
    "Retired": 0.257376,
    "Employed": 0.250567,
    "Unemployed": 0.248917,
    "Self-Employed": 0.243140
}

# Get total number of rows in cleaned_data
total_rows = len(cleaned_data)

# Count how many records we currently have
current_counts = Counter(cleaned_data["Employment_Status"])


def fix_employment_smart(row):
    age = row["Age"]
    current_status = row["Employment_Status"]

    # Is the age valid for this status?
    min_age, max_age = employment_age_limits.get(current_status, (0, 150))
    if min_age <= age <= max_age:
        return current_status  # No fix needed

    # Find all valid statuses for this age
    valid_statuses = [
        status for status, (min_a, max_a) in employment_age_limits.items()
        if min_a <= age <= max_a
    ]

    if not valid_statuses:
        return current_status  # fallback

    # Prefer status that brings us closer to original proportions
    best_status = None
    smallest_gap = float("inf")

    for status in valid_statuses:
        current_prop = current_counts[status] / total_rows
        target_prop = target_distribution.get(status, 0)
        gap = abs(target_prop - current_prop)
        if gap < smallest_gap:
            smallest_gap = gap
            best_status = status

    # Update the count to reflect the new assignment
    current_counts[best_status] += 1

    return best_status

In [ ]:
#Apply the function to fix the logical inconsistencies in the "Employment_Status" column
cleaned_data["Employment_Status"] = cleaned_data.apply(fix_employment_smart, axis=1)

# Check final distribution
print(cleaned_data["Employment_Status"].value_counts(normalize=True))

In [ ]:
#Perform a test to validate that the above function worked
invalid_employment_status = cleaned_data[cleaned_data.apply(lambda row: not (employment_age_limits.get(row["Employment_Status"], (0, 150))[
                                              0] <= row["Age"] <= employment_age_limits.get(row["Employment_Status"], (0, 150))[1]), axis=1)]
if invalid_employment_status.empty:
    print("All employment statuses are valid based on the age ranges")
else:
    print(
        f"❌ Found {len(invalid_employment_status)} invalid rows where employment status does not match the age.")
    display(invalid_employment_status)

#### Number of years driving

* The `Years_Driving` column represents the number of years that a person has been driving. 
* Logically, the values in this column can't be larger than the person's age minus 18 as the minimum age to obtain a full driving licence is 18 in South Africa. 
* We will proceed to identify rows in the `Years_Driving` column that are logically inaccurate. 

In [ ]:
# Identify rows where years_driving is greater than (age - 18)
invalid_years_driving_rows = cleaned_data[cleaned_data["Years_Driving"] > (
    cleaned_data["Age"] - 18)]
print(invalid_years_driving_rows)

* There are 4573 logically inconsistent rows which account for nearly half of the total rows in the dataset so removing these rows will lead to significant loss of data. 
* We will use a bar graph to visualize the distribution of the logical inconsistencies across various age groups. 

In [ ]:
# Define age bins
bins = [18, 25, 35, 45, 55, 65, 75, 100]  # Adjust as needed
labels = ["18-25", "26-35", "36-45", "46-55",
          "56-65", "66-75", "76+"]  # Labels for bins

# Create a new column for age groups
invalid_years_driving_rows["Age_Group"] = pd.cut(
    invalid_years_driving_rows["Age"], bins=bins, labels=labels, right=True)

# Count invalid cases per age group
invalid_counts_grouped = invalid_years_driving_rows["Age_Group"].value_counts(
).sort_index()

# Plot bar chart
plt.figure(figsize=(10, 5))
ax = sns.barplot(x=invalid_counts_grouped.index,
                 y=invalid_counts_grouped.values, color="red")

# Add numbers on top of bars (convert to integer)
for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}',  # Convert to integer
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center',
                fontsize=12, color='black',
                xytext=(0, 5), textcoords='offset points')

plt.title("Count of Invalid Years Driving by Age Group")
plt.xlabel("Age Group")
plt.ylabel("Number of Invalid Entries")

plt.show()

* The `18-45` age groups have the top 3 highest logical inconsistencies.  

* We can set the `Years_Driving` to be equal to `Age - 18` if it exceeds that value. 
* This ensures that we do not lose any valuable data while keeping the data consistent.

In [ ]:
#Replace any logical inconsistent rows to be within valid range
cleaned_data.loc[cleaned_data["Years_Driving"] > (cleaned_data["Age"] - 18), "Years_Driving"] = cleaned_data["Age"] - 18

* We will now perform a test to determine that all the logical inconsistencies have been fixed by the above function:

In [ ]:
#Check whether the Years_Driving column is now within the valid range
invalid_years_driving_rows = cleaned_data[cleaned_data["Years_Driving"] > (
    cleaned_data["Age"] - 18)]
print(invalid_education_rows)

# Display a summary sentence
if invalid_years_driving_rows.empty:
    print("✅ All employment statuses are valid based on the age ranges.")
else:
    print(
        f"❌ Found {len(invalid_years_driving_rows)} invalid rows where employment status does not match the age.")
    display(invalid_years_driving_rows)

#### Median insurance premium amount per annum by car details

In [ ]:
# Group by 'Car_Model', 'Car_Make', 'Manufacture_Year' and calculate the median of 'Premium_Amount'
median_model_premiums = cleaned_data.groupby(
    ["Car_Model", "Car_Make", "Manufacture_Year"])["Premium_Amount"].median().reset_index()

# Create a new column that combines Car_Model, Car_Make, and Manufacture_Year
median_model_premiums['Car_Description'] = (median_model_premiums['Car_Model'] + " " +
                                            median_model_premiums['Car_Make'] + " (" +
                                            median_model_premiums['Manufacture_Year'].astype(str) + ")")

# Sort by 'Premium_Amount' in descending order and select top 5
top_5_models = median_model_premiums.sort_values(
    by="Premium_Amount", ascending=False).head(5)

# Create the bar plot
plt.figure(figsize=(10, 6))
ax = sns.barplot(y="Car_Description", x="Premium_Amount",
                 data=top_5_models, palette="bright")

# Add value labels on top of the bars
for index, value in enumerate(top_5_models["Premium_Amount"]):
    plt.text(value, index, f'{value:,.0f}', va="center", fontsize=10)

plt.ylabel("Car Description (Model, Make, Year)")
plt.xlabel("Median Premium Amount per annum")
plt.title("Median Insurance Premium per annum by Car Model, Make, and Year")
plt.show()

The horizontal bar graph displays the top 5 cars with the highest median insurance premium per annum using model, make and year.

* The **Hyundai Tucson(1999)** has the highest median annual premium at **R83,295**. 
    * Older cars sometimes have higher premiums due to expensive parts which may not be readily avaliable or increased risk factors.

* The **Hyundai Elantra(2012)** has the lowest among the top five, at **R77,256**.
    * This can be as a result of better safety features, lower theft rates, or lower repair costs compared to others. 

* The **Kia Forte(2022)** has a high premium(**R83,892**) despite being the newest while the **Civic(1998)** and **CR-V(2002)** also have similar premiums. 
    * This suggests that other factors such as claim history, accident history play a role beyond car age. 

The median premiums are relatively close, implying that factors beyond car type might have a stronger impact. 

#### Distribution of insurance claims by the number of accidents

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x="Number_of_Accidents", y="Number_of_Claims",
            data=cleaned_data, palette="pastel")
plt.xlabel("Number of Accidents")
plt.ylabel("Number of Claims")
plt.title("Distribution of Claims by Number of Accidents")
plt.show()

The above visual is a box plots showing the distribution of the number of claims across different number of accident categories. 

* The number of claims does not increase linearly with the number of accidents - which is unexpcted for an insurance dataset. 
* This means that other factors such as policy term, years driving may influence clam numbers more than accident count.
* This also suggests that claims may not be strongly correlated with the number of accidents, or at least not in a simple, direct way. 
* Claims can include other incidents besides accidents.
* This could be an indication of potential fraudelent claims, unrreporting of accidents or policyholders have multiple claims from a single accident. 

#### Median premium amount per annum by the number of accidents and claims

In [ ]:
heatmap_data = cleaned_data.pivot_table(
    index="Number_of_Claims",
    columns="Number_of_Accidents",
    values="Premium_Amount",
    aggfunc="median"
)

plt.figure(figsize=(8, 6))
sns.heatmap(heatmap_data, cmap="coolwarm", annot=True, fmt=".0f", linewidths=0.5, cbar_kws={"label": "Median Premium Amount per annum"})

plt.xlabel("Number of Accidents")
plt.ylabel("Number of Claims")
plt.title("Median Premium Amount per annum by Number of Accidents and Claims")
plt.show()

The heatmap above displays the median premium amount per annum according to *claim history* and *accident history*. 

* There is no postive linear relationship between the claim history and the accident history which is surprising as one is expected to pay more premiums as the number of claims increase because your risk profile also increases. 
* The highest premium per annum (**R57,996**) has had only one accident and three claims which could be a result of claims from other incidents such as stolen vehicle parts, a weather-related claim or a glass-replacement claim.
* The second highest premium per annum(**R57,681**) has had only six claims and two accidents which are higher than the indivdual with the highest premium.
    * This suggests that other factors may play a bigger role in determining the final insurance premium. 
* The lowest premium per annum (**R44,334**) is from an individual with four accidents and nine claims which is surprising. This can due to a number of reasons: 
    * Long term discounts based on customer loyalty or multiple policies by the same insurer
    * Low-risk driver profile based on demographic factors
    * The claim types are not-at-fault



#### Medium premium amount per annum by years_of_driving

In [ ]:
#Visualize the median premium per annum based on the number of years drivibg 
median_years_premiums = cleaned_data.groupby("Years_Driving")["Premium_Amount"].median().reset_index()
plt.figure(figsize=(8, 6))
sns.lineplot(x="Years_Driving", y="Premium_Amount", data=median_years_premiums, color="b")
plt.xlabel("Years of Driving")
plt.ylabel("Median Premium Amount per annum")
plt.title("Median Amount by Years of Driving")
plt.grid(True)
plt.show()

The above line graph displays the median premium amount per annum by years of driving.

* The higest premium amount is from an individual with 20 years of driving experience.
* The lowest premium amount is from an individual with 60 years driving experience. 
* There are varying lengths of premiums amounts as the years of driving increase which could be to a number of reasons:
    * Non-linear risk patterns
    * Claim history vs driving experience
    * Differences in policy terms
    * Insurance company pricing as a result of age, loyalty discounts or market changes. 

There is typically no linear relationship between the two variables indicating that there might be various other factors which have an impact on the final premium amount.

#### Credit score range vs Median premium amount per annum


In [ ]:
#Sort the credit scores into categories 
def categorize_credit_score(credit_score):
    if credit_score < 580:
        return "Poor"
    elif 580 <= credit_score < 670:
        return "Fair"
    elif 670 <= credit_score < 740:
        return "Good"
    elif 740 <= credit_score < 800:
        return "Very Good"
    else:
        return "Excellent"


#Apply the function to the main dataset
cleaned_data["Credit_Category"] = cleaned_data["Credit_Score"].apply(
    categorize_credit_score)

#Calculate median premiums for each category
median_premium = cleaned_data.groupby("Credit_Category")[
    "Premium_Amount"].median().sort_values(ascending=False)

#Reorder categories in the dataset based on sorted order
sorted_categories = median_premium.index.tolist()
cleaned_data["Credit_Category"] = pd.Categorical(
    cleaned_data["Credit_Category"],
    categories=sorted_categories,
    ordered=True
)

#Plot using original cleaned_data but sorted categories
plt.figure(figsize=(10, 6))
sns.barplot(
    data=cleaned_data,
    y="Credit_Category",
    x="Premium_Amount",
    estimator="median",
    order=sorted_categories,
    palette="coolwarm",
    ci=None
)

#Add labels to the horizontal bars
for i, (cat, val) in enumerate(median_premium.items()):
    plt.text(val + 20, i, f"{int(val)}", va="center", fontsize=12)

plt.title("Median Premium by Credit Score Category")
plt.ylabel("Credit Score Category")
plt.xlabel("Median Premium (ZAR)")
plt.xlim(0, median_premium.max() + 100)
plt.tight_layout()
plt.show()

The above graph is a horizontal bar graph that represents the median premium amount across six different credit score categories. 

* The results are surprising as one would usually expect the following: 
    * Higher credit score would lead to lower premiums, because insurers usually consiser good credit as a sign of lower risk. 
    * The *Excellent* and *Very Good* credit score categories have the same highest premiums while *Poor* and *Good* have the lowest premiums. 
* Possible explanations for the above observations could be: 
    * Other features dominate risk assessment
    * Correlation with other features - e.g people with excellent credit might be insuring more expensive cars

#### Median premium amount per annum vs anti-theft device

In [ ]:
# Group by Anti-Theft Device status and calculate median premium
median_premium_anti_theft = cleaned_data.groupby("Has_AntiTheft_Device")[
    "Premium_Amount"].median().reset_index()

# Sort in descending order (optional)
median_premium_anti_theft = median_premium_anti_theft.sort_values(
    "Premium_Amount", ascending=False)

# Convert the column to categorical for proper ordering
median_premium_anti_theft["Has_AntiTheft_Device"] = median_premium_anti_theft["Has_AntiTheft_Device"].map({
                                                                                                    1: "Yes", 0: "No"})

plt.figure(figsize=(8, 5))

# Create horizontal bar chart
sns.barplot(y="Has_AntiTheft_Device", x="Premium_Amount",
            data=median_premium_anti_theft, palette="coolwarm")

# Add labels with thousands separator
for index, row in median_premium_anti_theft.iterrows():
    plt.text(row.Premium_Amount + 50, index,  # Offset label for readability
             f"{row.Premium_Amount:,.0f}",  # Format numbers with commas
             ha="left", va="center", fontsize=12, color="black")

plt.title("Median Premium Amount: Anti-Theft Device vs No Device", fontsize=14)
plt.ylabel("Anti-Theft Device Installed?", fontsize=12)
plt.xlabel("Median Premium (ZAR)", fontsize=12)

# Adjust x-axis limits for better spacing
plt.xlim(0, median_premium_anti_theft["Premium_Amount"].max() + 500)

# Light grid lines for readability
plt.grid(axis="x", linestyle="--", alpha=0.6)
plt.show()

The above graph is a categorical bar graph that displays the median premium per annum of individuals who have anti-theft device vs. those without it. 
* The median premiums are very similar which suggests that other features play a more significant role in determining the premium.
* Typically, having an anti-theft device lowers your risk profile as it improves vehicle security, aids in recovery when stolen thus potentially reducing insurance costs. 

#### Correlation Heatmap

In [ ]:
numeric_train_columns = cleaned_data.select_dtypes(include=[np.number])
correlation_matrix = numeric_train_columns.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", cbar=True, linewidths=0.5)
plt.title("Correlation Heatmap of Numeric Features", fontsize=16)
plt.show()

The above visual represents the correlation heatmap of the numeric features in our dataset.
* There is a positive moderate correlation between the age and the years_driving feature. 
    * This means that as the age increases, the years_driving column also increases(or vice versa)
    * This is due to the fact that the years_driving feature is dependent on age it measures the number of years an individual has been driving since they turned 18. 
* There is no correlation across all the other features indicating that there exists a non-linear relationship. 

### Five-number summary

In [ ]:
cleaned_data.describe()

* The five-number summary above provides a quick summary of the dataset. These include the minimum, first quartile, median, third quartile, and maximum. 
* This helps identify any potential outliners and gives insights into the data's central tendency and spread.

<a id="four"></a>
## ⚙️ 4. Feature Engineering
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Transforming our data ⚡ |
| :--------------------------- |

---